In [ ]:
import xmlrpc.client
import os

api_url = os.environ.get('ODOO_URL_API')
api_db = os.environ.get('ODOO_DB_API')
api_username = os.environ.get('ODOO_USERNAME_API')
api_clave = os.environ.get('ODOO_CLAVE_API')

common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
uid = common.authenticate(api_db, api_username, api_clave, {})
models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')

Obtener datos del Json del pago

In [ ]:
payment_id = 1555

[payment_json] = models.execute_kw(api_db, uid, api_clave, 'account.payment', 'read', [[payment_id]])
payment_json

No avances si alguno de los siguientes keys es False o una lista vacía

In [ ]:
print(payment_json['move_id'][0])
print(payment_json['attachment_ids'])
print(payment_json['l10n_mx_edi_payment_document_ids'])
print(payment_json['l10n_mx_edi_document_ids'])

Encontrar el json de edi_payment_document o generarlo si no existe

In [ ]:
[some_fact_w_edi] = models.execute_kw(api_db, uid, api_clave, 'account.move', 'read', [[86937]])

some_fact_w_edi['l10n_mx_edi_document_ids']

In [ ]:
edi_document_ids = 25296

In [ ]:
models.execute_kw(api_db, uid, api_clave, 'l10n_mx_edi.document', 'read', [[edi_document_ids]])

Buscar el attachment en el "Bucket"

In [ ]:
attachment_founds = models.execute_kw(api_db, uid, api_clave, 'ir.attachment', 'search', [[("name", "ilike", "PP13")]])
attachment_founds

In [ ]:
id_attachment = 89410

In [ ]:
[attachment_json] = models.execute_kw(api_db, uid, api_clave, 'ir.attachment', 'read', [id_attachment])
attachment_json

Corregir el Json del attachment encontrado

In [ ]:
text_index_content = attachment_json['raw'][:-1]

pos = text_index_content.find('UUID=')
if pos == -1:
    print('corrige el "uuid"')
    uuid = False
else:
    uuid = text_index_content[pos+6:pos+5+37]

In [ ]:
attachment_keys_to_modify = {
  'description': 'CFDI',
  'res_model': 'account.move',
  'res_id': payment_json['move_id'][0],
  'mimetype': 'text/plain',
  'index_content': text_index_content,
}    

models.execute_kw(api_db, uid, api_clave, 'ir.attachment', 'write', [[id_attachment], attachment_keys_to_modify])

Corrige json del payment

In [ ]:
payment_keys_to_modify = {'l10n_mx_edi_payment_document_ids': [edi_document_ids],
        'l10n_mx_edi_document_ids': [edi_document_ids],
        'l10n_mx_edi_cfdi_attachment_id': [attachment_json['id'], attachment_json['name']],
        'attachment_ids': [id_attachment]
    }

models.execute_kw(api_db, uid, api_clave, 'account.payment', 'write', [[payment_id], payment_keys_to_modify])

In [ ]:
models.execute_kw(api_db, uid, api_clave, 'l10n_mx_edi.document', 'read', [[edi_document_ids]])

In [ ]:
edi_document_keys_to_modify = {
        'attachment_uuid': uuid
    }

models.execute_kw(api_db, uid, api_clave, 'l10n_mx_edi.document', 'write', [[edi_document_ids], edi_document_keys_to_modify])